In [1]:

from transformers import pipeline
import warnings
import requests
import os
import streamlit as st
import ollama

warnings.filterwarnings('ignore')
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

/Users/supriyasindigerekumaraswmamy/Desktop/Thesis/wind_Turbine/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-07 13:23:27.686987: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
# new prompt
def shap_tranform(scenario):
    prompt = f"""You are an intelligent assistant designed to explain complex machine learning predictions and their visualizations, like SHAP waterfall plots, in simple and easy-to-understand terms for users with no background in machine learning. Your main goal is to explain why the model made a specific prediction and provide actionable insights based on the SHAP values.


Context:

We are analyzing data from a wind farm, focusing on the performance of five turbines (T01, T06, T07, T09, T11). 
Each turbine is equipped with various sensors that collect data every 10 minutes over two years. 
The dataset includes sensor data (81 variables like generator speed, pitch angle, wind speed), meteorological data (40 variables including temperature and humidity), 
and failure data for critical components (gearbox, transformer, generator bearing, generator, hydraulic group).

Our machine learning model predicts whether a turbine component (such as the gearbox or generator bearing) is faulty or non-faulty. 
To make these predictions more understandable, we use SHAP (SHapley Additive exPlanations), 
a method that explains how much each feature in the data contributes to a particular prediction.


Task:

Receive Input: You will receive SHAP values shap values used in the waterfall plot in the waterfall plot lower shapvalues are not shown and only the 10 features with high shap values are shown.
Transform Output: Your job is to explain the SHAP waterfall plot and its key insights in a simple way. Help the user understand:
            -- Explain what is the prediction ML model made for the instance by considering target class input provided.  
            -- How to interpret the overall contribution of features to the model’s prediction, explaining their impact on the turbine component's health.
How to interpret the input: The input consists of SHAP values for different features of a turbine component. Positive SHAP values indicate features that contribute to a faulty prediction, while negative SHAP values indicate features that contribute to a non-faulty prediction. The input also consists of target class which specifies the prediction of the instance.

Output Structure:

Explain about What is SHAP and waterfall plots of shap? explain what is postive prediction mean in our case and what is negative prediction mean in our case.
Explain the SHAP Waterfall Plot: Describe the SHAP waterfall plot and its components in simple terms. Explain that the features with high negative SHAP values contribute to a non-faulty prediction, while features with high positive SHAP values contribute to a faulty prediction.
Actionable Insights: Explain which factors are pushing the prediction of that instance by considering the target class value. If the prediction is non-faulty, explain the top 3 features with the highest negative SHAP values (i.e., the features that contributed the most to keeping the prediction non-faulty). If the prediction is faulty, explain the top 3 features with the highest positive SHAP values (i.e., the features that contributed the most to the failure prediction.
Conclusion: Provide a brief summary that helps the user understand the implications of the prediction and suggested actions.

Explain all the above points in simple terms that a non-technical user can understand in 5-10 sentences. 


Input:
{scenario}

Transformed Explanation:
"""
    response = ollama.chat(model='llama3', options={'temperature': 0}, messages=[{
        'role': 'user',
        'content': prompt,
    }])
    explanation = response['message']['content']
    return explanation

In [ ]:
#-- Which features contributed to a non-faulty prediction (negative SHAP values).
#Explain which factors are pushing the prediction of that instance by considering the target class value. What it means if the instance has more negative shapley values or more postive shapley values and how these values are pushing the model prediction of the instance.

In [21]:
scenario = [{"target_class":" Faulty"}, {
               "generator_rotations_per_minute_min": 0.646938145160675,
                "generator_bearing_temperature_average": -0.7935078740119934,
                "minimum_ambient_wind_speed": 0.011441145092248917,
                "average_absolute_ambient_wind_direction": 1.546500325202942,
                "average_ambient_temperature": 1.4875788688659668,
                "average_controller_top_temperature": -0.3365827202796936,
                "average_controller_hub_temperature": -0.3822290599346161,
                "average_grid_rotor_inverter_phase_2_temperature": 0.21342462301254272,
                "average_grid_rotor_inverter_phase_3_temperature": 2.0366032123565674,
                "grid_production_frequency_average": -0.11417296528816223,
                "grid_production_power_maximum": 0.10816410928964615,
                "generator_bearing2_temperature_average": -0.6663039326667786,
                "max_windspeed3": 0.4138451814651489
            }]
story = shap_tranform(scenario)

In [22]:
print(story)

Here's a simple explanation of the SHAP waterfall plot and its insights:

**What is SHAP?**
SHAP (SHapley Additive exPlanations) is a method that explains how much each feature in our data contributes to a particular prediction. It helps us understand why the machine learning model made a specific prediction.

**Waterfall Plot:**
The SHAP waterfall plot shows the features that contributed most to the prediction. The plot has two main parts: positive and negative contributions. Features with high **negative** values contribute to a non-faulty prediction, while features with high **positive** values contribute to a faulty prediction.

**Prediction Insights:**
For this instance, the target class is "Faulty", which means the model predicts that the turbine component is faulty. Let's look at the top 3 features that contributed most to this prediction:

* The average grid rotor inverter phase 2 temperature had a high positive SHAP value, indicating it was one of the main factors contributing

In [6]:
def generate_story(scenario):
    prompt = f"""You are an AI assistant that transforms technical outputs from explainable AI (XAI) algorithms into a format that is easy for humans to understand. Your goal is to provide clear, concise, and informative explanations of XAI outputs, making them accessible to individuals without technical backgrounds.

Instructions:

Component Explanation: Describe the component mentioned in the output, explaining its role and importance.
Anchor Condition: Simplify the condition specified by the anchor, explaining what it means in practical terms.
Precision: Explain what the precision value indicates about the model's predictions.
Coverage: Describe what the coverage value signifies regarding the model’s applicability.
Overall Implications: Summarize the implications of these values for understanding the model’s behavior and potential actions.

Format:

Component: Explanation of the component's role and importance.
Anchor Condition: Simplified explanation of the condition.
Precision: Explanation of the precision value.
Coverage: Explanation of the coverage value.
Overall Implications: Summary of the implications.

Input:
{scenario}

Transformed Explanation:
"""
    response = ollama.chat(model='llama3', options={'temperature': 0}, messages=[{
        'role': 'user',
        'content': prompt,
    }])
    story = response['message']['content']
    print(story)
    return story

In [ ]:
# old prompt

def shap_tranform(scenario):
    prompt = f"""You are an intelligent assistant designed to explain complex machine learning predictions and their visualizations, like SHAP waterfall plots, in simple and easy-to-understand terms for users with no background in machine learning.


Context:

We are analyzing data from a wind farm, focusing on the performance of five turbines (T01, T06, T07, T09, T11). 
Each turbine is equipped with various sensors that collect data every 10 minutes over two years. 
The dataset includes sensor data (81 variables like generator speed, pitch angle, wind speed), meteorological data (40 variables including temperature and humidity), 
and failure data for critical components (gearbox, transformer, generator bearing, generator, hydraulic group).

Our machine learning model predicts whether a turbine component (such as the gearbox or generator bearing) is faulty or non-faulty. 
To make these predictions more understandable, we use SHAP (SHapley Additive exPlanations), 
a method that explains how much each feature in the data contributes to a particular prediction.


Task:

Receive Input: You will receive SHAP values shap values used in the waterfall plot in the waterfall plot lower shapvalues are not shown and only the 10 features with high shap values are shown.
Transform Output: Your job is to explain the SHAP waterfall plot and its key insights in a simple way. Help the user understand:
            -- Which features contributed to a faulty prediction (positive SHAP values).
            -- Which features contributed to a non-faulty prediction (negative SHAP values).
            -- How to interpret the overall contribution of features to the model’s prediction, explaining their impact on the turbine component's health.
How to interpret the input: The input consists of SHAP values for different features of a turbine component. Positive SHAP values indicate features that contribute to a faulty prediction, while negative SHAP values indicate features that contribute to a non-faulty prediction. The input also consists of target class which specifies the prediction of the instance.

Output Structure:

Explain about What is SHAP and waterfall plots of shap? explain what is postive prediction mean in our case and what is negative prediction mean in our case.
Explain the SHAP Waterfall Plot: Describe the SHAP waterfall plot and its components in simple terms. the input consists of all the shap values used in the plot and lower shapvalues are not shown and only the 10 features with high shap values are shown. 
Actionable Insights: Explain which factors are pushing the prediction of that instance. What it means if the instance has more negative shapley values or more postive shapley values and how these values are pushing the model prediction of the instance.
Conclusion: Provide a brief summary that helps the user understand the implications of the prediction and suggested actions.


Input:
{scenario}

Transformed Explanation:
"""
    response = ollama.chat(model='llama3', options={'temperature': 0}, messages=[{
        'role': 'user',
        'content': prompt,
    }])
    explanation = response['message']['content']
    return explanation